In [87]:
!pip install -q python-terrier==0.11.0
!pip install elasticsearch

In [88]:
#Imported directly from kaggle
import pandas as pd
#Use the reduced dataset for timing and capacity issues 
df = pd.read_csv("/kaggle/input/recipe-sampled-0-25/sampled_dataset.csv")

In [89]:
import pandas as pd
import pyterrier as pt
from elasticsearch import Elasticsearch

if not pt.started():
  pt.init()

/tmp/ipykernel_31/3645034949.py:5: DeprecationWarning: Call to deprecated function (or staticmethod) started. (use pt.java.started() instead) -- Deprecated since version 0.11.0.
  if not pt.started():


**Index by ingredients**

In [90]:
import ast

def ingrs_to_list_str(list_):
    list_of_string = []
    for item in list_:
        toStr = ' '.join(ast.literal_eval(item))
        list_of_string.append(toStr)
    return list_of_string

colSeries = df['ingredients']
ingredients_list = ingrs_to_list_str(colSeries)

toEx = ingredients_list[:10]

for i in toEx:
    print(f"\n\n {i}")



 1 1/2 pound flank steak 1/2 c. finely minced green onions (scallions) 1/2 c. dry red wine 1/4 c. soy sauce 3 tbsp. salad oil 3 teaspoon sesame seeds 2 teaspoon packed brown sugar 1/4 teaspoon grnd black pepper 1/4 teaspoon grnd ginger 1 clove garlic, chopped


 1 tablespoon rosemary 1 teaspoon thyme 3 bay leaves 1 teaspoon smoked paprika 1 teaspoon pepper 1/4 cup red wine 3 cups chicken broth 2 cups button mushrooms sliced 2 cups mushroom mix, oyster, shiitake, baby bella, sliced 2 medium carrots sliced diagonally 1 onion medium, chopped 1 red potato medium, cut in 1-inch pieces 1 cup frozen green beans 1-inch pieces 1/2 can black olives pitted ripe, halved 1 handful grape tomatoes halved 8 chicken thighs with bones and skin. 2-3 lbs 2 stalks celery 3 cups water


 3 to 4 carrots 1 1/2 Tbsp. butter 1/3 c. brown sugar grated lemon rind and juice


 4.5 Cups Flour 1.5 Tsp Salt Pinch Baking Powder 1 Tbls Sugar 1 2/3 cup Crisco 1 egg lightly beaten 1 tsp vinegar Ice Water


 2 c. crushe

In [91]:
documents = [{'docno': str(i), 'text': text} for i, text in enumerate(ingredients_list)]
indexer = pt.IterDictIndexer("./index", overwrite=True)
indexer.index(documents)

21:03:32.500 [ForkJoinPool-4-worker-3] WARN org.terrier.structures.indexing.Indexer -- Adding an empty document to the index (159129) - further warnings are suppressed
21:04:15.191 [ForkJoinPool-4-worker-3] WARN org.terrier.structures.indexing.Indexer -- Indexed 7 empty documents


<org.terrier.querying.IndexRef at 0x796bc8b7c890 jclass=org/terrier/querying/IndexRef jself=<LocalRef obj=0x2f21ebe8 at 0x796be17ed9f0>>

In [92]:
index = pt.IndexFactory.of("./index")

#Classic index
bm25 = pt.terrier.Retriever(index, wmodel="BM25")
tfidf = pt.terrier.Retriever(index, wmodel="TF_IDF")

#Specialized index for this contex

#In this case the repetition doesn't count
dfree = pt.terrier.Retriever(index, wmodel="DFRee")
# k1=0 force BM25 to ignore term frequency and only count matches.
bm25_no_tf = pt.terrier.Retriever(index, wmodel="BM25", controls={"k_1": "0.0", "b": "0.0"})


In [93]:
print(index.getCollectionStatistics().toString())

Number of documents: 557658
Number of terms: 29085
Number of postings: 14945695
Number of fields: 1
Number of tokens: 22660675
Field names: [text]
Positions:   false



In [94]:
#Some queries 

queries = [
    {"qid": "1", "query": "chicken"},
    {"qid": "2", "query": "tomato basil"},
    {"qid": "3", "query": "rice chicken onion"},
    {"qid": "4", "query": "beef onion garlic pepper"},
    {"qid": "5", "query": "salmon lemon dill butter"},
    {"qid": "6", "query": "pasta olive oil garlic cheese"},
    {"qid": "7", "query": "mushrooms cream parmesan thyme"},
    {"qid": "8", "query": "potatoes butter rosemary salt"},
    {"qid": "9", "query": "tuna celery mayonnaise pickle"},
    {"qid": "10", "query": "zucchini olive oil pepper onion"},
    {"qid": "11", "query": "apple cinnamon sugar flour"},
    {"qid": "12", "query": "banana oats honey peanut butter"},
    {"qid": "13", "query": "chocolate butter flour eggs sugar"},
    {"qid": "14", "query": "shrimp garlic lemon parsley"},
    {"qid": "15", "query": "broccoli cheese cream garlic"},
]

In [95]:
for i, q in enumerate(queries):
    if i%2==0:
        print(bcolors.OKGREEN)
    else:
        print(bcolors.OKBLUE)
    
    result_bm25 = bm25.search(q['query'])
    result_tfidf = tfidf.search(q['query'])
    result_dfree = dfree.search(q['query'])
    result_bm25_no_tf = bm25_no_tf.search(q['query'])

    print(result_bm25)
    print(result_tfidf)
    print(result_dfree)
    print(result_bm25_no_tf)



    qid   docid   docno  rank     score    query
0     1   21793   21793     0  5.089820  chicken
1     1  213014  213014     1  5.069910  chicken
2     1  113411  113411     2  5.034422  chicken
3     1  226140  226140     3  5.010803  chicken
4     1  545842  545842     4  5.010803  chicken
..   ..     ...     ...   ...       ...      ...
995   1  298145  298145   995  4.401338  chicken
996   1  345964  345964   996  4.401338  chicken
997   1  350860  350860   997  4.401338  chicken
998   1  352596  352596   998  4.401338  chicken
999   1  384829  384829   999  4.401338  chicken

[1000 rows x 6 columns]
    qid   docid   docno  rank     score    query
0     1   21793   21793     0  3.179007  chicken
1     1  213014  213014     1  3.166571  chicken
2     1  113411  113411     2  3.144406  chicken
3     1  226140  226140     3  3.129654  chicken
4     1  545842  545842     4  3.129654  chicken
..   ..     ...     ...   ...       ...      ...
995   1  298145  298145   995  2.748993  ch

In [96]:
titles = df['title']

for i, q in enumerate(queries):
    print(f"\n=== QUERY {i+1}: {q} ===")

    result_bm25 = bm25.search(q['query'])
    result_tfidf = tfidf.search(q['query'])
    result_dfree = dfree.search(q['query'])
    result_bm25_no_tf = bm25_no_tf.search(q['query'])

    for method_name, result in [
        ("BM25", result_bm25),
        ("TF-IDF", result_tfidf),
        ("DFRee", result_dfree),
        ("BM25_no_tf", result_bm25_no_tf),
    ]:
        print(f"\n--- MODELLO: {method_name} ---")
        
        # Limita i risultati a massimo 5
        for docno in result["docno"][:5]:  # <-- TAGLIA qui i risultati
            docno_int = int(docno)  # forza conversione str -> int
            title = titles.iloc[docno_int] if docno_int < len(titles) else "TITLE NOT FOUND"
            print(f"Docno: {docno} | Title: {title}")




=== QUERY 1: {'qid': '1', 'query': 'chicken'} ===

--- MODELLO: BM25 ---
Docno: 21793 | Title: Cheesy Bacon Chicken
Docno: 213014 | Title: Baked Chicken Recipe
Docno: 113411 | Title: Perfect Roast Chicken Recipe
Docno: 226140 | Title: Chicken Pie
Docno: 545842 | Title: Chicken Casserole

--- MODELLO: TF-IDF ---
Docno: 21793 | Title: Cheesy Bacon Chicken
Docno: 213014 | Title: Baked Chicken Recipe
Docno: 113411 | Title: Perfect Roast Chicken Recipe
Docno: 226140 | Title: Chicken Pie
Docno: 545842 | Title: Chicken Casserole

--- MODELLO: DFRee ---
Docno: 551102 | Title: Chicken Pot Pie(350~ Oven)  
Docno: 337182 | Title: Chicken And Dumplings
Docno: 389820 | Title: Chicken Casserole
Docno: 554398 | Title: Chicken Dumpling Casserole
Docno: 112653 | Title: Napoleon Chicken

--- MODELLO: BM25_no_tf ---
Docno: 21793 | Title: Cheesy Bacon Chicken
Docno: 213014 | Title: Baked Chicken Recipe
Docno: 113411 | Title: Perfect Roast Chicken Recipe
Docno: 226140 | Title: Chicken Pie
Docno: 545842 | 

Consideration:

The retrieval results are overall very good.
- BM25, TF-IDF, and BM25_no_tf consistently retrieved highly relevant recipes, especially when queries matched common ingredients.
- DFRee performed slightly more varied but still reasonable, it differnts a little bit from the other

There is strong consistency between BM25 and TF-IDF, especially for simple queries.
In general, the indexing and retrieval setup works well for recipe search based on ingredients.

**Index by title**

In [97]:
coltitle = df['title'].tolist()

toEx = coltitle[3000:3010]

for i in toEx:
    print(f"\n\n {i}")



 Calico Salad


 Tofu Herb Spread


 Egg Salad With A Twist


 Pork Braised In Milk And Cream Recipe


 Gluten Free Rosemary Bread 


 Hawaiian Grilled Pork Chops


 Easy Christmas Cupcake Decorations


 Bibimbap/Rice Mixed With Vegetables And Beef 


 Deluxe Peas


 Donna B's Meatballs


In [101]:
documents = [{'docno': str(i), 'text': text} for i, text in enumerate(coltitle)]
indexer = pt.IterDictIndexer("./index_title", overwrite=True)
indexer.index(documents)

21:05:44.323 [ForkJoinPool-7-worker-3] WARN org.terrier.structures.indexing.Indexer -- Adding an empty document to the index (194) - further warnings are suppressed
21:06:05.281 [ForkJoinPool-7-worker-3] WARN org.terrier.structures.indexing.Indexer -- Indexed 46 empty documents


<org.terrier.querying.IndexRef at 0x796be30df110 jclass=org/terrier/querying/IndexRef jself=<LocalRef obj=0x2f21e4f0 at 0x796be1bbb950>>

In [102]:
index = pt.IndexFactory.of("./index_title")

#Classic index
bm25 = pt.terrier.Retriever(index, wmodel="BM25")
tfidf = pt.terrier.Retriever(index, wmodel="TF_IDF")

#Specialized index for this contex
dfree = pt.terrier.Retriever(index, wmodel="DFRee")
bm25_no_tf = pt.terrier.Retriever(index, wmodel="BM25", controls={"k_1": "0.0", "b": "0.0"})

In [103]:
print(index.getCollectionStatistics().toString())

Number of documents: 557658
Number of terms: 34052
Number of postings: 1858185
Number of fields: 1
Number of tokens: 1866522
Field names: [text]
Positions:   false



In [104]:
#Some queries 

queries = [
    {"qid": "1", "query": "chicken"},
    {"qid": "2", "query": "tomato"},
    {"qid": "3", "query": "rice"},
    {"qid": "4", "query": "beef"},
    {"qid": "5", "query": "salmon lemon"},
    {"qid": "6", "query": "pasta"},
    {"qid": "7", "query": "Cornmeal Rolls"},
    {"qid": "8", "query": "Meatballs"},
    {"qid": "9", "query": "tuna celery mayonnaise pickle"},
    {"qid": "10", "query": "Cupcake"},
    {"qid": "11", "query": "Fettuccine"},
    {"qid": "12", "query": "banana"},
    {"qid": "13", "query": "Spiced Wieners"},
    {"qid": "14", "query": "Cookies"},
    {"qid": "15", "query": "Hamburger And Potatoes"},
]

In [105]:
for i, q in enumerate(queries):
    if i%2==0:
        print(bcolors.OKGREEN)
    else:
        print(bcolors.OKBLUE)
    
    result_bm25 = bm25.search(q['query'])
    result_tfidf = tfidf.search(q['query'])
    result_dfree = dfree.search(q['query'])
    result_bm25_no_tf = bm25_no_tf.search(q['query'])

    print(result_bm25)
    print(result_tfidf)
    print(result_dfree)
    print(result_bm25_no_tf)


    qid   docid   docno  rank     score    query
0     1    6952    6952     0  5.021262  chicken
1     1    6986    6986     1  5.021262  chicken
2     1  231410  231410     2  5.021262  chicken
3     1  245792  245792     3  5.021262  chicken
4     1  345320  345320     4  5.021262  chicken
..   ..     ...     ...   ...       ...      ...
995   1   59654   59654   995  4.244088  chicken
996   1   59681   59681   996  4.244088  chicken
997   1   59692   59692   997  4.244088  chicken
998   1   59740   59740   998  4.244088  chicken
999   1   59815   59815   999  4.244088  chicken

[1000 rows x 6 columns]
    qid   docid   docno  rank     score    query
0     1    6952    6952     0  2.915111  chicken
1     1    6986    6986     1  2.915111  chicken
2     1  231410  231410     2  2.915111  chicken
3     1  245792  245792     3  2.915111  chicken
4     1  345320  345320     4  2.915111  chicken
..   ..     ...     ...   ...       ...      ...
995   1   59654   59654   995  2.463920  ch

In [106]:
titles = df['title']

for i, q in enumerate(queries):
    print(f"\n=== QUERY {i+1}: {q} ===")

    result_bm25 = bm25.search(q['query'])
    result_tfidf = tfidf.search(q['query'])
    result_dfree = dfree.search(q['query'])
    result_bm25_no_tf = bm25_no_tf.search(q['query'])

    for method_name, result in [
        ("BM25", result_bm25),
        ("TF-IDF", result_tfidf),
        ("DFRee", result_dfree),
        ("BM25_no_tf", result_bm25_no_tf),
    ]:
        print(f"\n--- MODELLO: {method_name} ---")
        
        # Limita i risultati a massimo 5
        for docno in result["docno"][:5]:  # <-- TAGLIA qui i risultati
            docno_int = int(docno)  # forza conversione str -> int
            title = titles.iloc[docno_int] if docno_int < len(titles) else "TITLE NOT FOUND"
            print(f"Docno: {docno} | Title: {title}")


=== QUERY 1: {'qid': '1', 'query': 'chicken'} ===

--- MODELLO: BM25 ---
Docno: 6952 | Title: Chicken Fried Chicken
Docno: 6986 | Title: Chicken With  Chicken With Lemon 
Docno: 231410 | Title: City Chicken (But It'S Not Chicken!)
Docno: 245792 | Title: Chicken Fried Chicken
Docno: 345320 | Title: Chicken Casserole With No Chicken

--- MODELLO: TF-IDF ---
Docno: 6952 | Title: Chicken Fried Chicken
Docno: 6986 | Title: Chicken With  Chicken With Lemon 
Docno: 231410 | Title: City Chicken (But It'S Not Chicken!)
Docno: 245792 | Title: Chicken Fried Chicken
Docno: 345320 | Title: Chicken Casserole With No Chicken

--- MODELLO: DFRee ---
Docno: 53975 | Title: Popeye Tso'S Chicken (General Tso'S Chicken Made With Popeye'S Chicken Nuggets) Recipe
Docno: 336299 | Title: Brandied Tarragon Cream Chicken (Chicken a la Vallee d'Auge), Camembert Rice with Warm Endive and Apples
Docno: 279696 | Title: Indian Green Bell Pepper And Chicken Curry: Home Style Chicken Jalfrezi
Docno: 2138 | Title: New 

The retrieval results are generally very good and consistent across models.

Possible implementation:
- merging searches by title and ingredients
- Inclusion of ingredients on hand and second check that they are indeed all present in those extracted
  